# Installs

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# Imports

In [2]:
import pandas as pd
import ast
from scipy.stats import ttest_ind

In [3]:
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import BertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

In [4]:
import sys
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [5]:
import numpy as np

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import Model

In [8]:
sys.path.append('model/code-bert/')
from temporal_relation_classification import TemporalRelationClassification
from temporal_relation_classification_config import TemporalRelationClassificationConfig

In [11]:
model_path = "saved_models/bert-base-uncased-saved-model"
model = TemporalRelationClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30526, 768)

# Original

## Import Original Datasets

In [239]:
occlusion = pd.read_csv('results/occlusion/occlusion-og-bert-base.csv')
gradients = pd.read_csv('results/gradients/gradients-og-bert-base.csv')

In [240]:
original = pd.read_csv('data/annotated/partitions.csv')
counterfactuals = pd.read_csv('data/annotated/counterfactuals.csv')

In [241]:
original_predictions = pd.read_csv('results/predictions/predictions-og-bert-base.csv')
counterfactual_predictions = pd.read_csv('results/predictions/predictions-cf-bert-base.csv')

## Process Datasets

In [242]:
import re
def annotate_text(row, column):
    context = row['context']
    eventA = re.escape(row['eventA'])
    eventB = re.escape(row['eventB'])

    if len(eventA) > len(eventB):
        context = re.sub(eventA, f"[a1]{row['eventA']}[/a1]", context)
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
    else:
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
        context = re.sub(eventA, f"[a1]{row['eventB']}[/a1]", context)

    return context

In [243]:
import re
def annotate_text_counter(row, column):
    context = row['counterfactual']
    eventA = re.escape(row['eventA'])
    eventB = re.escape(row['eventB'])

    # Make sure the longer event is replaced first if they overlap
    if len(eventA) > len(eventB):
        context = re.sub(eventA, f"[a1]{row['eventA']}[/a1]", context)
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
    else:
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
        context = re.sub(eventA, f"[a1]{row['eventB']}[/a1]", context)

    return context

In [244]:
original['annotated_context'] = original.apply(lambda row: annotate_text(row, 'context'), axis=1)
counterfactuals['annotated_context'] = counterfactuals.apply(lambda row: annotate_text_counter(row, 'counterfactual'), axis=1)

In [245]:
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    "partition": Dataset.from_pandas(original),
    "counterfactual":  Dataset.from_pandas(counterfactuals)
})

In [246]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenizer_class = str(type(tokenizer)).strip("><'").split('.')[-1]

In [247]:
def preprocess_function(examples):
  max_length_value = 508
  return tokenizer(examples["annotated_context"], truncation=True, max_length=max_length_value)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [250]:
original['index'] = range(len(original))
counterfactual['index'] = range(len(counterfactual))
original_predictions['index'] = range(len(original_predictions))
counterfactual_predictions['index'] = range(len(counterfactual_predictions))
occlusion['index'] = range(len(occlusion))
gradients['index'] = range(len(gradients))

In [252]:
merged = original
merged['counterfactual'] = counterfactual['counterfactual']
merged['pos_partition_counterfactual'] = counterfactual['pos_partition']
merged['label_counterfactual'] = counterfactual['new_label']
merged['occlusion_saliency'] = occlusion['word_attributions']
merged['occlusion_saliency'] = occlusion['word_attributions']
merged['gradient_saliency'] = gradients['word_attributions']
merged['original_predictions'] = original_predictions['prediction']
merged['counterfactual_predictions'] = counterfactual_predictions['prediction']

In [253]:
merged['index'] = range(len(merged))

In [254]:
merged.columns

Index(['id', 'context', 'eventA', 'eventB', 'label_temp', 'label',
       'pos_partition', 'annotated_context', 'input_ids', 'token_type_ids',
       'attention_mask', 'index', 'counterfactual',
       'pos_partition_counterfactual', 'label_counterfactual',
       'occlusion_saliency', 'gradient_saliency', 'original_predictions',
       'counterfactual_predictions'],
      dtype='object')

## Process Partition + Counterfactuals

In [255]:
check_accuracy_original = (merged['original_predictions'] == merged['label']).mean()
check_accuracy_counterfactual = (merged['counterfactual_predictions'] == merged['label_counterfactual']).mean()

In [256]:
merged['counterfactual_correct'] = merged['label_counterfactual'] == merged['counterfactual_predictions']
merged['original_correct'] = merged['label'] == merged['original_predictions']

In [257]:
count_true_counterfactuals = (merged['counterfactual_correct']).sum()
count_true_original = (merged['original_correct']).sum()
count_both_true = (merged['counterfactual_correct'] & merged['original_correct']).sum()
print("Number of instances where counterfactual true :", count_true_counterfactuals)
print("Number of instances where original true :", count_true_original)
print("Number of instances where both counterfactual_correct and original_correct are True:", count_both_true)
print("Percentage of instances where both hold:", count_both_true/len(counterfactuals))

Number of instances where counterfactual true : 80
Number of instances where original true : 111
Number of instances where both counterfactual_correct and original_correct are True: 33
Percentage of instances where both hold: 0.1625615763546798


In [258]:
df = merged[merged['counterfactual_correct'] & merged['original_correct']]

## Check for: Original + Occlusion

In [134]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for token, scores in attrs:
            avg_score = sum(scores) / len(scores) if scores else 0  # Compute average score, handle empty lists
            rows.append({'index': idx, 'token': token, 'score': avg_score})
    return pd.DataFrame(rows)

In [135]:
def get_saliency_partition_df(df_method, df_partition):
    saliency_partition = pd.merge(df_method, df_partition, on='index', how='left')
    saliency_partition['partition'] = saliency_partition.apply(determine_partition, axis=1)
    saliency_partition = saliency_partition[['index', 'token', 'score', 'partition']]
    return saliency_partition

In [136]:
def apply_ttest(group):
    pos_scores = group[group['partition'] == 1]['score']
    neg_scores = group[group['partition'] == 0]['score']
    if len(pos_scores) > 1 and len(neg_scores) > 1:
        stat, p_value = ttest_ind(pos_scores, neg_scores, alternative='greater')
        return pd.Series({'t_stat': stat, 'p_value': p_value})
    else:
        return pd.Series({'t_stat': np.nan, 'p_value': np.nan})

def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [137]:
def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [138]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for attr in attrs:
            if isinstance(attr, tuple) and len(attr) == 2:
                token, scores = attr
                if scores and isinstance(scores, list): 
                    avg_score = sum(scores) / len(scores)  
                else:
                    avg_score = 0  
                rows.append({'index': idx, 'token': token, 'score': avg_score})
            else:
                print(f"Unexpected format in attributions: {attr}")
    return pd.DataFrame(rows)

In [140]:
def determine_partition_original(row):
    special_tokens = {'[SEP]', '[CLS]', '[a1]', '[a2]', '[/a1]', '[/a2]'}
    if row['token'] in row['original_pos_partition']: 
        return 1
    else:
        return 0  

In [141]:
def parse_list_string(s):
    s = s.strip("[]")
    items = s.split("', '")
    if items:
        items[0] = items[0].strip("'")
        items[-1] = items[-1].strip("'")
    return items

In [142]:
def convert_to_list(string):
    string = string.strip('{}')
    return string.split(', ')

In [234]:
def convert_to_list(s):
    s = s.strip('{}')  # Remove the curly braces
    return [item.strip() for item in s.split(',')]

In [259]:
occlusion['word_attributions'] = occlusion['word_attributions'].apply(lambda x: ast.literal_eval(x.replace("('[", "(\"[").replace("]',", "]\",").replace(")']", ")]")))
saliency_occlusion = get_saliency_df(occlusion, 'index', 'word_attributions')

In [260]:
saliency_occlusion = df.merge(saliency_occlusion, on='index', how='left')

In [276]:
occlusion_partition = get_saliency_partition_df(saliency_occlusion, partitions)
saliency_occlusion['partition'] = saliency_occlusion.apply(determine_partition, axis=1)
saliency_partition = saliency_occlusion[['index', 'token', 'score', 'partition']]

In [277]:
saliencies = df.merge(saliency_partition, on='index', how='left')

## Check for: Original + Gradients

In [155]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for token, scores in attrs:
            avg_score = sum(scores) / len(scores) if scores else 0  # Compute average score, handle empty lists
            rows.append({'index': idx, 'token': token, 'score': avg_score})
    return pd.DataFrame(rows)

In [156]:
def get_saliency_partition_df(df_method, df_partition):
    saliency_partition = pd.merge(df_method, df_partition, on='index', how='left')
    saliency_partition['partition'] = saliency_partition.apply(determine_partition, axis=1)
    saliency_partition = saliency_partition[['index', 'token', 'score', 'partition']]
    return saliency_partition

In [157]:
def apply_ttest(group):
    pos_scores = group[group['partition'] == 1]['score']
    neg_scores = group[group['partition'] == 0]['score']
    if len(pos_scores) > 1 and len(neg_scores) > 1:
        stat, p_value = ttest_ind(pos_scores, neg_scores, alternative='greater')
        return pd.Series({'t_stat': stat, 'p_value': p_value})
    else:
        return pd.Series({'t_stat': np.nan, 'p_value': np.nan})

def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [158]:
def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [159]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for attr in attrs:
            if isinstance(attr, tuple) and len(attr) == 2:
                token, scores = attr
                rows.append({'index': idx, 'token': token, 'score': scores})
            else:
                print(f"Unexpected format in attributions: {attr}")
    return pd.DataFrame(rows)

In [160]:
def determine_partition(row):
    special_tokens = {'[SEP]', '[CLS]', '[a1]', '[a2]', '[/a1]', '[/a2]'}
    if row['token'] in row['pos_partition']: 
        return 1
    else:
        return 0  

In [161]:
def determine_partition_original(row):
    special_tokens = {'[SEP]', '[CLS]', '[a1]', '[a2]', '[/a1]', '[/a2]'}
    if row['token'] in row['original_pos_partition']: 
        return 1
    else:
        return 0  

In [162]:
def parse_list_string(s):
    s = s.strip("[]")
    items = s.split("', '")
    if items:
        items[0] = items[0].strip("'")
        items[-1] = items[-1].strip("'")
    return items

In [163]:
def convert_to_list(string):
    string = string.strip('{}')
    return string.split(', ')

In [284]:
gradients['word_attributions'] = gradients['word_attributions'].apply(lambda x: ast.literal_eval(x.replace("('[", "(\"[").replace("]',", "]\",").replace(")']", ")]")))
saliency_gradients = get_saliency_df(gradients, 'index', 'word_attributions')
gradients_partition = get_saliency_partition_df(saliency_gradients, original)

In [285]:
gradients_df = df.merge(gradients_partition, on='index', how='left')

In [289]:
gradients_df['pos_partition'] = gradients_df['pos_partition'].apply(convert_to_list)
gradients_df['pos_partition'] = gradients_df.apply(determine_partition, axis=1)
results = compute_significance_partition(gradients_df)

In [290]:
count_true = (results['significantly_higher'] == True).sum()
ratio_true = (count_true/len(results))

In [291]:
print(f"Proportion of instances in occlusion, significantly higher: ", {ratio_true})

Proportion of instances in occlusion, significantly higher:  {0.3939393939393939}


In [292]:
print(f"Number of instances in occlusion, significantly higher: " , {count_true})

Number of instances in occlusion, significantly higher:  {13}


# Counterfactuals

## Import Original Datasets

In [239]:
occlusion = pd.read_csv('results/occlusion/occlusion-cf-bert-base.csv')
gradients = pd.read_csv('results/gradients/gradients-cf-bert-base.csv')

In [240]:
original = pd.read_csv('data/annotated/partitions.csv')
counterfactuals = pd.read_csv('data/annotated/counterfactuals.csv')

In [241]:
original_predictions = pd.read_csv('results/predictions/predictions-og-bert-base.csv')
counterfactual_predictions = pd.read_csv('results/predictions/predictions-cf-bert-base.csv')

## Process Datasets

In [242]:
import re
def annotate_text(row, column):
    context = row['context']
    eventA = re.escape(row['eventA'])
    eventB = re.escape(row['eventB'])

    if len(eventA) > len(eventB):
        context = re.sub(eventA, f"[a1]{row['eventA']}[/a1]", context)
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
    else:
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
        context = re.sub(eventA, f"[a1]{row['eventB']}[/a1]", context)

    return context

In [243]:
import re
def annotate_text_counter(row, column):
    context = row['counterfactual']
    eventA = re.escape(row['eventA'])
    eventB = re.escape(row['eventB'])

    # Make sure the longer event is replaced first if they overlap
    if len(eventA) > len(eventB):
        context = re.sub(eventA, f"[a1]{row['eventA']}[/a1]", context)
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
    else:
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
        context = re.sub(eventA, f"[a1]{row['eventB']}[/a1]", context)

    return context

In [244]:
original['annotated_context'] = original.apply(lambda row: annotate_text(row, 'context'), axis=1)
counterfactuals['annotated_context'] = counterfactuals.apply(lambda row: annotate_text_counter(row, 'counterfactual'), axis=1)

In [245]:
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    "partition": Dataset.from_pandas(original),
    "counterfactual":  Dataset.from_pandas(counterfactuals)
})

In [246]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenizer_class = str(type(tokenizer)).strip("><'").split('.')[-1]

In [247]:
def preprocess_function(examples):
  max_length_value = 508
  return tokenizer(examples["annotated_context"], truncation=True, max_length=max_length_value)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [250]:
original['index'] = range(len(original))
counterfactual['index'] = range(len(counterfactual))
original_predictions['index'] = range(len(original_predictions))
counterfactual_predictions['index'] = range(len(counterfactual_predictions))
occlusion['index'] = range(len(occlusion))
gradients['index'] = range(len(gradients))

In [252]:
merged = original
merged['counterfactual'] = counterfactual['counterfactual']
merged['pos_partition_counterfactual'] = counterfactual['pos_partition']
merged['label_counterfactual'] = counterfactual['new_label']
merged['occlusion_saliency'] = occlusion['word_attributions']
merged['occlusion_saliency'] = occlusion['word_attributions']
merged['gradient_saliency'] = gradients['word_attributions']
merged['original_predictions'] = original_predictions['prediction']
merged['counterfactual_predictions'] = counterfactual_predictions['prediction']

In [253]:
merged['index'] = range(len(merged))

In [254]:
merged.columns

Index(['id', 'context', 'eventA', 'eventB', 'label_temp', 'label',
       'pos_partition', 'annotated_context', 'input_ids', 'token_type_ids',
       'attention_mask', 'index', 'counterfactual',
       'pos_partition_counterfactual', 'label_counterfactual',
       'occlusion_saliency', 'gradient_saliency', 'original_predictions',
       'counterfactual_predictions'],
      dtype='object')

## Process Partition + Counterfactuals

In [255]:
check_accuracy_original = (merged['original_predictions'] == merged['label']).mean()
check_accuracy_counterfactual = (merged['counterfactual_predictions'] == merged['label_counterfactual']).mean()

In [256]:
merged['counterfactual_correct'] = merged['label_counterfactual'] == merged['counterfactual_predictions']
merged['original_correct'] = merged['label'] == merged['original_predictions']

In [257]:
count_true_counterfactuals = (merged['counterfactual_correct']).sum()
count_true_original = (merged['original_correct']).sum()
count_both_true = (merged['counterfactual_correct'] & merged['original_correct']).sum()
print("Number of instances where counterfactual true :", count_true_counterfactuals)
print("Number of instances where original true :", count_true_original)
print("Number of instances where both counterfactual_correct and original_correct are True:", count_both_true)
print("Percentage of instances where both hold:", count_both_true/len(counterfactuals))

Number of instances where counterfactual true : 80
Number of instances where original true : 111
Number of instances where both counterfactual_correct and original_correct are True: 33
Percentage of instances where both hold: 0.1625615763546798


In [258]:
df = merged[merged['counterfactual_correct'] & merged['original_correct']]

## Check for: Counterfactuals + Occlusion

In [134]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for token, scores in attrs:
            avg_score = sum(scores) / len(scores) if scores else 0  # Compute average score, handle empty lists
            rows.append({'index': idx, 'token': token, 'score': avg_score})
    return pd.DataFrame(rows)

In [135]:
def get_saliency_partition_df(df_method, df_partition):
    saliency_partition = pd.merge(df_method, df_partition, on='index', how='left')
    saliency_partition['partition'] = saliency_partition.apply(determine_partition, axis=1)
    saliency_partition = saliency_partition[['index', 'token', 'score', 'partition']]
    return saliency_partition

In [136]:
def apply_ttest(group):
    pos_scores = group[group['partition'] == 1]['score']
    neg_scores = group[group['partition'] == 0]['score']
    if len(pos_scores) > 1 and len(neg_scores) > 1:
        stat, p_value = ttest_ind(pos_scores, neg_scores, alternative='greater')
        return pd.Series({'t_stat': stat, 'p_value': p_value})
    else:
        return pd.Series({'t_stat': np.nan, 'p_value': np.nan})

def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [137]:
def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [138]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for attr in attrs:
            if isinstance(attr, tuple) and len(attr) == 2:
                token, scores = attr
                if scores and isinstance(scores, list): 
                    avg_score = sum(scores) / len(scores)  
                else:
                    avg_score = 0  
                rows.append({'index': idx, 'token': token, 'score': avg_score})
            else:
                print(f"Unexpected format in attributions: {attr}")
    return pd.DataFrame(rows)

In [140]:
def determine_partition_original(row):
    special_tokens = {'[SEP]', '[CLS]', '[a1]', '[a2]', '[/a1]', '[/a2]'}
    if row['token'] in row['original_pos_partition']: 
        return 1
    else:
        return 0  

In [141]:
def parse_list_string(s):
    s = s.strip("[]")
    items = s.split("', '")
    if items:
        items[0] = items[0].strip("'")
        items[-1] = items[-1].strip("'")
    return items

In [142]:
def convert_to_list(string):
    string = string.strip('{}')
    return string.split(', ')

In [234]:
def convert_to_list(s):
    s = s.strip('{}')  # Remove the curly braces
    return [item.strip() for item in s.split(',')]

In [259]:
occlusion['word_attributions'] = occlusion['word_attributions'].apply(lambda x: ast.literal_eval(x.replace("('[", "(\"[").replace("]',", "]\",").replace(")']", ")]")))
saliency_occlusion = get_saliency_df(occlusion, 'index', 'word_attributions')

In [260]:
saliency_occlusion = df.merge(saliency_occlusion, on='index', how='left')

In [276]:
occlusion_partition = get_saliency_partition_df(saliency_occlusion, partitions)
saliency_occlusion['partition'] = saliency_occlusion.apply(determine_partition, axis=1)
saliency_partition = saliency_occlusion[['index', 'token', 'score', 'partition']]

In [277]:
saliencies = df.merge(saliency_partition, on='index', how='left')

## Check for: Counterfactuals + Gradients

In [155]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for token, scores in attrs:
            avg_score = sum(scores) / len(scores) if scores else 0  # Compute average score, handle empty lists
            rows.append({'index': idx, 'token': token, 'score': avg_score})
    return pd.DataFrame(rows)

In [156]:
def get_saliency_partition_df(df_method, df_partition):
    saliency_partition = pd.merge(df_method, df_partition, on='index', how='left')
    saliency_partition['partition'] = saliency_partition.apply(determine_partition, axis=1)
    saliency_partition = saliency_partition[['index', 'token', 'score', 'partition']]
    return saliency_partition

In [157]:
def apply_ttest(group):
    pos_scores = group[group['partition'] == 1]['score']
    neg_scores = group[group['partition'] == 0]['score']
    if len(pos_scores) > 1 and len(neg_scores) > 1:
        stat, p_value = ttest_ind(pos_scores, neg_scores, alternative='greater')
        return pd.Series({'t_stat': stat, 'p_value': p_value})
    else:
        return pd.Series({'t_stat': np.nan, 'p_value': np.nan})

def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [158]:
def compute_significance_partition(df_saliency_partition):
    results = df_saliency_partition.groupby('index').apply(apply_ttest)
    results['significantly_higher'] = (results['p_value'] < 0.05)
    return results

In [159]:
def get_saliency_df(df, index_col, attributions_col):
    rows = []
    for idx, attrs in zip(df[index_col], df[attributions_col]):
        for attr in attrs:
            if isinstance(attr, tuple) and len(attr) == 2:
                token, scores = attr
                rows.append({'index': idx, 'token': token, 'score': scores})
            else:
                print(f"Unexpected format in attributions: {attr}")
    return pd.DataFrame(rows)

In [160]:
def determine_partition(row):
    special_tokens = {'[SEP]', '[CLS]', '[a1]', '[a2]', '[/a1]', '[/a2]'}
    if row['token'] in row['pos_partition']: 
        return 1
    else:
        return 0  

In [161]:
def determine_partition_original(row):
    special_tokens = {'[SEP]', '[CLS]', '[a1]', '[a2]', '[/a1]', '[/a2]'}
    if row['token'] in row['original_pos_partition']: 
        return 1
    else:
        return 0  

In [162]:
def parse_list_string(s):
    s = s.strip("[]")
    items = s.split("', '")
    if items:
        items[0] = items[0].strip("'")
        items[-1] = items[-1].strip("'")
    return items

In [163]:
def convert_to_list(string):
    string = string.strip('{}')
    return string.split(', ')

In [284]:
gradients['word_attributions'] = gradients['word_attributions'].apply(lambda x: ast.literal_eval(x.replace("('[", "(\"[").replace("]',", "]\",").replace(")']", ")]")))
saliency_gradients = get_saliency_df(gradients, 'index', 'word_attributions')
gradients_partition = get_saliency_partition_df(saliency_gradients, original)

In [285]:
gradients_df = df.merge(gradients_partition, on='index', how='left')

In [289]:
gradients_df['pos_partition'] = gradients_df['pos_partition'].apply(convert_to_list)
gradients_df['pos_partition'] = gradients_df.apply(determine_partition, axis=1)
results = compute_significance_partition(gradients_df)

In [290]:
count_true = (results['significantly_higher'] == True).sum()
ratio_true = (count_true/len(results))

In [291]:
print(f"Proportion of instances in occlusion, significantly higher: ", {ratio_true})

Proportion of instances in occlusion, significantly higher:  {0.3939393939393939}


In [292]:
print(f"Number of instances in occlusion, significantly higher: " , {count_true})

Number of instances in occlusion, significantly higher:  {13}
